<a href="https://colab.research.google.com/github/tatiana-iazykova/2020_HACK_RUSSIANSUPERGLUE/blob/main/HA_Topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install pymorphy2 gensim nltk
!pip install pyLDAvis
import gensim
import json
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim
import string
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Image
from IPython.core.display import HTML 
morph = MorphAnalyzer()

In [2]:
%%capture
!wget https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/wiki_data.txt.zip
!unzip wiki_data.txt.zip

Основная задача - **построить хорошую тематическую модель с интерпретируемыми топиками с помощью LDA в gensim и NMF в sklearn**.


1) сделайте нормализацию (если pymorphy2 работает долго используйте mystem или попробуйте установить быструю версию - `pip install pymorphy2[fast]`, можно использовать какой-то другой токенизатор); 

2) добавьте нграммы (в тетрадке есть закомменченая ячейка с Phrases,  можно также попробовать другие способы построить нграммы); 

3) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова); 

4) постройте несколько LDA моделей (переберите количество тем, можете поменять alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем; 

5) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы;

6) между словарем и обучением модели добавьте tfidf (`tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary); corpus = tfidf[corpus]`);

7) повторите пункт 4 на преобразованном корпусе;

8) в отдельной ячейке опишите как изменилась модель (приведите несколько тем, которые стали лучше или хуже, или которых раньше вообще не было; можно привести значения перплексии и когерентности для обеих моделей)

9) проделайте такие же действия для NMF (образец в конце тетрадки), для построения словаря воспользуйтесь возможностями Count или Tfidf Vectorizer (попробуйте другие значение max_features, min_df, max_df, сделайте нграмы через ngram_range, если хватает памяти), попробуйте такие же количества тем

10) в отдельной ячейке напечатайте темы лучшей NMF модели, сравните их с теми, что получились в LDA.

Сохраните тетрадку с экспериментами и положите её на гитхаб, ссылку на неё укажите в форме.

**Оцениваться будут главным образом пункты 5, 8 и 10. (2, 3, 2 баллов соответственно). Чтобы заработать остальные 3 балла, нужно хотя бы немного изменить мой код на промежуточных этапах (добавить что-то, указать другие параметры и т.д). **

# 1. Нормализация

In [5]:
import nltk
nltk.download('stopwords')
stops = set(stopwords.words('russian') + ["это", "весь"])
# чтобы быстрее нормализовать тексты, создадим словарь всех словоформ
# нормазуем каждую 1 раз и положим в словарь
# затем пройдем по текстам и сопоставим каждой словоформе её нормальную форму

def opt_normalize(texts, top=None):
    uniq = Counter()
    for text in texts:
        uniq.update(text)
    
    norm_uniq = {word:morph.parse(word)[0].normal_form for word, _ in uniq.most_common(top)}
    
    norm_texts = []
    for text in texts:
        
        norm_words = [norm_uniq.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stops]
        norm_texts.append(norm_words)
        
    return norm_texts

def tokenize(text):
    tokens = re.findall('[а-яёa-z0-9]+', text.lower())
    words = [word for word in tokens if len(word) > 2]   
    return words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
texts_init = open('wiki_data.txt').read().splitlines()[:20000]

In [7]:
texts = opt_normalize([tokenize(text) for text in texts_init], 50000)

# 2. Н-граммы

In [ ]:
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.2)
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

In [ ]:
ngrammed_texts[16]

# 3. Построение словаря

In [ ]:
dictinary = gensim.corpora.Dictionary(ngrammed_texts)
dictinary.filter_extremes(no_above=0.3, no_below=50)
dictinary.compactify()

In [ ]:
corpus = [dictinary.doc2bow(text) for text in ngrammed_texts]

# 4. Постройте LDA моделей

In [ ]:
lda = gensim.models.LdaMulticore(corpus, 200, id2word=dictinary, eval_every=10, passes=20)

In [ ]:
lda.print_topics()

[(132,
  '0.114*"свет" + 0.074*"внутренний" + 0.062*"ядро" + 0.049*"мозг" + 0.046*"глаз" + 0.032*"сторона" + 0.031*"оболочка" + 0.026*"нос" + 0.026*"110" + 0.026*"лука"'),
 (198,
  '0.162*"норвегия" + 0.119*"норвежский" + 0.088*"латвия" + 0.085*"шведский" + 0.048*"перу" + 0.035*"римский_папа" + 0.033*"южный_америка" + 0.026*"плоский" + 0.026*"склон" + 0.021*"собранный"'),
 (94,
  '0.093*"литовский" + 0.084*"литва" + 0.064*"1964_год" + 0.063*"лев" + 0.055*"воевода" + 0.052*"1964" + 0.050*"иванович" + 0.046*"василиевич" + 0.042*"1937" + 0.040*"1953"'),
 (38,
  '0.116*"система" + 0.032*"управление" + 0.030*"устройство" + 0.023*"оборудование" + 0.022*"использоваться" + 0.021*"также" + 0.018*"работа" + 0.017*"блок" + 0.017*"механизм" + 0.015*"тип"'),
 (31,
  '0.017*"персонаж" + 0.017*"являться" + 0.013*"свой" + 0.013*"человек" + 0.011*"также" + 0.011*"время" + 0.011*"мочь" + 0.010*"мир" + 0.009*"сила" + 0.009*"враг"'),
 (53,
  '0.091*"император" + 0.057*"империя" + 0.053*"царь" + 0.042*"хан

In [ ]:
import numpy as np
lda.log_perplexity(corpus)

-7.984884112548162

In [ ]:
coherence_model_lda = gensim.models.CoherenceModel(model=lda, 
                                                  texts=ngrammed_texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [ ]:
topics = []
for topic_id, topic in lda.show_topics(num_topics=200, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

In [ ]:
coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=ngrammed_texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [ ]:
coherence_model_lda.get_coherence()

0.532579919490243

In [ ]:
lda_1 = gensim.models.LdaMulticore(corpus, 200, id2word=dictinary, eval_every=5, passes=5)

In [ ]:
lda_1.print_topics()

[(142,
  '0.065*"команда" + 0.033*"динамо" + 0.032*"матч" + 0.024*"принять_участие" + 0.021*"сентябрь" + 0.018*"выступать" + 0.017*"титул" + 0.016*"чемпион" + 0.015*"высокий_лига" + 0.013*"выступление"'),
 (51,
  '0.115*"2006" + 0.111*"2005_год" + 0.087*"2005" + 0.059*"счёт" + 0.055*"казанский" + 0.050*"декабрь_2009" + 0.048*"кубань" + 0.036*"андре" + 0.032*"январь_2006" + 0.026*"нидерландский"'),
 (151,
  '0.019*"бог" + 0.018*"герой" + 0.018*"картина" + 0.016*"мир" + 0.015*"пещера" + 0.011*"рука" + 0.011*"богиня" + 0.009*"находить" + 0.009*"сон" + 0.009*"изобразить"'),
 (161,
  '0.233*"зона" + 0.095*"пустынь" + 0.064*"раствор" + 0.061*"николаевский" + 0.057*"треугольник" + 0.056*"роза" + 0.041*"весёлый" + 0.031*"петровский" + 0.031*"подавать" + 0.031*"компонент"'),
 (190,
  '0.012*"мочь" + 0.009*"использовать" + 0.009*"также" + 0.009*"должный" + 0.009*"работа" + 0.008*"кроме" + 0.008*"большой" + 0.007*"хороший" + 0.007*"требовать" + 0.006*"сторона"'),
 (35,
  '0.048*"институт" + 0.039

In [ ]:
lda_1.log_perplexity(corpus)

-8.051139864724997

In [ ]:
coherence_model_lda_1 = gensim.models.CoherenceModel(model=lda_1, 
                                                  texts=ngrammed_texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [ ]:
topics_1 = []
for topic_id, topic in lda_1.show_topics(num_topics=200, formatted=False):
    topic = [word for word, _ in topic]
    topics_1.append(topic)

In [ ]:
coherence_model_lda_1 = gensim.models.CoherenceModel(topics=topics_1, 
                                                   texts=ngrammed_texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [ ]:
coherence_model_lda_1.get_coherence()

0.5067069935002525

In [ ]:
lda_2 = gensim.models.LdaMulticore(corpus, 100, id2word=dictinary, eval_every=2, passes=13)

In [ ]:
lda_2.print_topics()

[(75,
  '0.271*"сша" + 0.123*"американский" + 0.066*"великобритания" + 0.065*"нью_йорк" + 0.058*"джон" + 0.055*"британский" + 0.044*"австралия" + 0.031*"австралийский" + 0.022*"новый_зеландия" + 0.021*"колледж"'),
 (14,
  '0.142*"хутор" + 0.084*"район_ростовский" + 0.079*"норвегия_административный" + 0.076*"станица" + 0.067*"область_входить" + 0.058*"мартин" + 0.057*"казак" + 0.047*"состав_сельский" + 0.040*"поселение" + 0.038*"сельский_поселение"'),
 (55,
  '0.195*"пётр" + 0.110*"литва" + 0.070*"казанский" + 0.048*"воронеж" + 0.046*"воронежский" + 0.043*"воронежский_область" + 0.042*"дэвис" + 0.040*"французский_язык" + 0.035*"уфа" + 0.035*"местечко"'),
 (9,
  '0.224*"олимпийский_игра" + 0.082*"участие_летний" + 0.065*"история_завоевать" + 0.033*"женщина" + 0.031*"медаль_сборная" + 0.026*"страна_представлять" + 0.025*"завоевать_медаль" + 0.019*"свой_история" + 0.019*"медаль" + 0.016*"участие_зимний"'),
 (99,
  '0.026*"животное" + 0.023*"человек" + 0.023*"собака" + 0.015*"земля" + 0.012

In [ ]:
lda_2.log_perplexity(corpus)

-7.679684670450383

In [ ]:
coherence_model_lda_2 = gensim.models.CoherenceModel(model=lda_2, 
                                                  texts=ngrammed_texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [ ]:
topics_2 = []
for topic_id, topic in lda_2.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics_2.append(topic)

In [ ]:
coherence_model_lda_2 = gensim.models.CoherenceModel(topics=topics_2, 
                                                   texts=ngrammed_texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [ ]:
coherence_model_lda_2.get_coherence()

0.5917689276861122

# 5. 3 темы для самой лучшей модели 

Несмотря на то, что метрики у модели lda_1 были выше, я бы все-таки назвала лучшей модель: 

```lda_2 = gensim.models.LdaMulticore(corpus, 200, id2word=dictinary, eval_every=5, passes=5)```

1. (142,
  '0.065*"команда" + 0.033*"динамо" + 0.032*"матч" + 0.024*"принять_участие" + 0.021*"сентябрь" + 0.018*"выступать" + 0.017*"титул" + 0.016*"чемпион" + 0.015*"высокий_лига" + 0.013*"выступление"')
2. (35,
  '0.048*"институт" + 0.039*"университет" + 0.031*"образование" + 0.021*"академия" + 0.019*"студент" + 0.018*"кафедра" + 0.018*"обучение" + 0.017*"учебный" + 0.015*"центр" + 0.015*"факультет"')

3. (196,
  '0.168*"самолёт" + 0.055*"полёт" + 0.038*"экипаж" + 0.035*"аэродром" + 0.031*"авиационный" + 0.029*"пилот" + 0.027*"севастополь" + 0.026*"лётчик" + 0.025*"истребитель" + 0.024*"ввс"')


# 6. Посчитаем Tfidf

In [ ]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictinary)

In [ ]:
corpus = tfidf[corpus]

# 7. Построим модели

In [ ]:
lda_tfidf_3 = gensim.models.LdaMulticore(corpus, 200, id2word=dictinary, eval_every=5, passes=30)

In [ ]:
lda_tfidf_3.print_topics()

[(188,
  '0.171*"тяжёлый_атлетика" + 0.096*"греция" + 0.074*"который_мочь" + 0.071*"скотт" + 0.066*"джек" + 0.057*"грек" + 0.057*"1908" + 0.046*"анна" + 0.033*"попытка" + 0.025*"нация"'),
 (76,
  '0.064*"футбольный_клуб" + 0.042*"клуб" + 0.042*"высокий_лига" + 0.030*"домашний_матч" + 0.030*"кубок" + 0.030*"выступать" + 0.029*"2006" + 0.029*"сборная_ссср" + 0.029*"спорт_ссср" + 0.028*"заслужить_мастер"'),
 (185,
  '0.065*"медный" + 0.047*"рот" + 0.046*"состав_группа" + 0.046*"new" + 0.045*"барабан" + 0.043*"вокалист" + 0.043*"турне" + 0.036*"господь" + 0.035*"получить_награда" + 0.034*"младший"'),
 (162,
  '0.037*"обучение" + 0.034*"ответственность" + 0.031*"финансовый" + 0.028*"знание" + 0.026*"трудовой" + 0.022*"высокий_образование" + 0.019*"обмен" + 0.019*"услуга" + 0.019*"сектор" + 0.018*"задание"'),
 (132,
  '0.080*"ричард" + 0.071*"1932" + 0.066*"1932_год" + 0.065*"раб" + 0.060*"крым" + 0.058*"голландский" + 0.049*"1802_год" + 0.043*"родиться_январь" + 0.038*"румынский" + 0.037*"т

In [ ]:
lda_tfidf_3.log_perplexity(corpus)

-15.279384812686724

In [ ]:
coherence_model_lda_tfidf_3 = gensim.models.CoherenceModel(model=lda_tfidf_3, 
                                                  texts=ngrammed_texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [ ]:
topics_3 = []
for topic_id, topic in lda_tfidf_3.show_topics(num_topics=200, formatted=False):
    topic = [word for word, _ in topic]
    topics_3.append(topic)

In [ ]:
coherence_model_lda_tfidf_3  = gensim.models.CoherenceModel(topics=topics_3, 
                                                   texts=ngrammed_texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [ ]:
coherence_model_lda_tfidf_3.get_coherence()

0.43017173086835553

In [ ]:
lda_tfidf_4 = gensim.models.LdaMulticore(corpus, 100, id2word=dictinary, eval_every=5, passes=30)

In [ ]:
lda_tfidf_4.print_topics()

[(11,
  '0.017*"здание" + 0.011*"герб" + 0.011*"памятник" + 0.011*"музей" + 0.009*"двор" + 0.009*"церковь" + 0.009*"храм" + 0.007*"город" + 0.007*"архитектор" + 0.007*"башня"'),
 (83,
  '0.094*"фильм" + 0.032*"актёр" + 0.028*"актриса" + 0.026*"семейство_листоед" + 0.026*"театр" + 0.025*"роль" + 0.025*"режиссёр" + 0.023*"сериал" + 0.023*"кино" + 0.018*"телесериал"'),
 (21,
  '0.223*"список" + 0.090*"станица" + 0.062*"рыба" + 0.040*"степной" + 0.040*"казак" + 0.031*"тихий" + 0.030*"ступень" + 0.027*"полость" + 0.024*"приключение" + 0.023*"and"'),
 (37,
  '0.172*"сельский_поселение" + 0.077*"муниципальный_образование" + 0.060*"воронежский_область" + 0.053*"поселение_входить" + 0.047*"населить_пункт" + 0.042*"административный_центр" + 0.036*"городской_поселение" + 0.032*"динамо" + 0.032*"состав" + 0.030*"поселение"'),
 (84,
  '0.255*"летний_олимпийский" + 0.114*"свой_история" + 0.112*"принимать_участие" + 0.110*"завоевать_медаль" + 0.067*"сборная_страна" + 0.040*"игра_2004" + 0.039*"предст

In [ ]:
lda_tfidf_4.log_perplexity(corpus)

-12.728674932549573

In [ ]:
coherence_model_lda_tfidf_4 = gensim.models.CoherenceModel(model=lda_tfidf_4, 
                                                  texts=ngrammed_texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [ ]:
topics_4 = []
for topic_id, topic in lda_tfidf_4.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics_4.append(topic)

In [ ]:
coherence_model_lda_tfidf_4  = gensim.models.CoherenceModel(topics=topics_4, 
                                                   texts=ngrammed_texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [ ]:
coherence_model_lda_tfidf_4.get_coherence()

0.4713654826207568

In [ ]:
lda_tfidf_5 = gensim.models.LdaMulticore(corpus, 100, id2word=dictinary, eval_every=1, passes=40)

In [ ]:
lda_tfidf_5.print_topics()

[(63,
  '0.152*"остров" + 0.091*"топоним" + 0.050*"область_россия" + 0.037*"земля_франц" + 0.036*"остров_архипелаг" + 0.034*"район_архангельский" + 0.034*"метр" + 0.033*"хороший_спортсмен" + 0.024*"водохранилище" + 0.024*"расположить_юго"'),
 (60,
  '0.072*"месторождение" + 0.051*"генрих" + 0.042*"соль" + 0.040*"юар" + 0.034*"геологический" + 0.031*"часто_использоваться" + 0.029*"женский" + 0.026*"мужской" + 0.025*"тонна" + 0.025*"уэльс"'),
 (86,
  '0.017*"входить_сельский" + 0.017*"армия" + 0.016*"дивизия" + 0.014*"война" + 0.014*"войско" + 0.014*"корабль" + 0.011*"бой" + 0.010*"флот" + 0.010*"полка" + 0.008*"командир"'),
 (17,
  '0.105*"код_4148" + 0.058*"шотландский" + 0.046*"шотландия" + 0.045*"великобритания" + 0.041*"ник" + 0.040*"немецкий_язык" + 0.038*"энтони" + 0.037*"рейтинг" + 0.033*"1987" + 0.029*"фанат"'),
 (44,
  '0.017*"ольга" + 0.011*"белоруссия" + 0.010*"минск" + 0.010*"деятель" + 0.010*"белорусский" + 0.009*"1939" + 0.009*"арестовать" + 0.008*"дмитрий" + 0.008*"одесса

In [ ]:
lda_tfidf_5.log_perplexity(corpus)

-12.548233850893604

In [ ]:
coherence_model_lda_tfidf_5 = gensim.models.CoherenceModel(model=lda_tfidf_5, 
                                                  texts=ngrammed_texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [ ]:
topics_5 = []
for topic_id, topic in lda_tfidf_5.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics_5.append(topic)

In [ ]:
coherence_model_lda_tfidf_5  = gensim.models.CoherenceModel(topics=topics_5, 
                                                   texts=ngrammed_texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [ ]:
coherence_model_lda_tfidf_5.get_coherence()

0.4756731957975824

# 8. Сравним модели

In [ ]:
models = ['lda', 'lda_1', 'lda_2', 'lda_tfidf_3', 'lda_tfidf_4', 'lda_tfidf_5']
perpl = [-7.984884113,	-8.051139865,	-7.67968467,	-15.27938481,	-12.72867493, -12.548233850893604]
coh = [0.5325799195,	0.5067069935,	0.5917689277,	0.4301717309,	0.4713654826, 0.4756731957975824]
df_metrics = pd.DataFrame({'model': models, 'perplexity': perpl, 'coherence': coh})
df_metrics

,model,perplexity,coherence
0,lda,-7.984884,0.532580
1,lda_1,-8.051140,0.506707
2,lda_2,-7.679685,0.591769
3,lda_tfidf_3,-15.279385,0.430172
4,lda_tfidf_4,-12.728675,0.471365
5,lda_tfidf_5,-12.548234,0.475673


Если приглядеться, то во всех шести моделях можно заметить, что темы очень сильно зависят от выбранных параметров. Однако можно выделить основные: 
1. олимпийские игры
2. перепись населения
3. гонки
4. исторические события
5. памятники
6. биология/эволюция

Для примера давайте рассмотрим тему переписи населения.

Она присутсвует в модели lda как
(169,
  '0.088*"расстояние_расположить" + 0.083*"харьковский_область" + 0.066*"район_харьковский" + 0.058*"ниже_течение" + 0.053*"перепись_2001" + 0.051*"коатуа_население" + 0.050*"год_составлять" + 0.050*"сельский_совет" + 0.049*"берег_река" + 0.048*"составлять_человек"')

  и 

  (147,
  '0.124*"житомирский_область" + 0.088*"село_украина" + 0.087*"район_житомирский" + 0.082*"год_составлять" + 0.081*"перепись_2001" + 0.073*"человек_почтовый" + 0.066*"область_код" + 0.066*"коатуа_население" + 0.064*"занимать_площадь" + 0.052*"тело"')

  В модели lda_tfidf_4:
  
  (66,
  '0.094*"перепись_2001" + 0.094*"район_житомирский" + 0.092*"год_составлять" + 0.089*"житомирский_область" + 0.088*"село_украина" + 0.082*"область_код" + 0.082*"коатуа_население" + 0.075*"человек_почтовый" + 0.059*"занимать_площадь" + 0.055*"индекс_телефонный"')

  В модели lda_tfidf_5:

  (76,
  '0.147*"район_удмуртия" + 0.075*"деревня_завьяловский" + 0.075*"центр_ижевск" + 0.059*"перепись_население" + 0.057*"миноносец" + 0.036*"департамент" + 0.032*"северо_восток" + 0.031*"вятский_губерния" + 0.030*"новый" + 0.025*"певец"')


В моделях lda и lda_tfidf_4 ключевые слова практически совпадают, однако в модели lda_tfidf_4, как мне кажется, слово с наибольшим коэффициентом лучше описывает всю тему, чем в модели lda

Что касается результатов метрик, то можно заметить, что модели без tfidf показали лучшие результаты, однако была замечена корреляция, что результаты моделей с tfidf увеличиваются пропорционально параметру passes 

# 9. NMF

In [8]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [9]:
stexts = [' '.join(text) for text in texts]

In [124]:
vectorizer = TfidfVectorizer(max_features=1000, min_df=0.1, max_df=0.2, norm= 'l2', ngram_range = (1, 2))
X = vectorizer.fit_transform(stexts)

In [125]:
vectorizer1 = CountVectorizer(max_features=1000, min_df=0.1, max_df=0.2, ngram_range = (1, 2))
X1 = vectorizer1.fit_transform(stexts)

In [137]:
# изменение параметра n_components вело к ухудшению результатов
model = NMF(n_components=100)

In [ ]:
model.fit(X)

In [ ]:
model1 = NMF(n_components=100)

In [ ]:
model1.fit(X1)

In [ ]:
model.reconstruction_err_

In [ ]:
model1.reconstruction_err_

In [132]:
feat_names = vectorizer.get_feature_names()

In [134]:
feat_names1 = vectorizer1.get_feature_names()

In [135]:
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 состав  январь  март  ещё
1 работа  январь  страна  основать
2 основать  страна  история  принять
3 принимать участие  игра  олимпийский  участие
4 город  страна  новый  расположить
5 река  история  война  октябрь
6 вид  украина  место  март
7 март  январь  украина  страна
8 получить  история  страна  октябрь
9 село  население  январь  основать
10 тело  население  конец  затем
11 украина  население  основать  главный
12 страна  игра  олимпийский  история
13 высокий  медаль  январь  страна
14 расположить  место  война  территория
15 состав  январь  октябрь  входить
16 новый  январь  мир  май
17 игра  олимпийский  медаль  история
18 однако  второй  история  январь
19 начало  октябрь  война  мир
20 работать  родиться  медаль  история
21 год составлять  занимать  перепись  составлять
22 группа  октябрь  принять  затем
23 город  май  мир  некоторый
24 мочь  принимать  история  страна
25 работа  история  страна  год составлять
26 2001 год  2001  январь  основать
27 война  медаль  январь  с

In [136]:
top_words1 = model1.components_.argsort()[:,:-5:-1]

for i in range(top_words1.shape[0]):
    words = [feat_names1[j] for j in top_words1[i]]
    print(i, "  ".join(words))

0 день  новый  январь  конец
1 иметь  начало  занимать  название
2 игра  2001 год  мир  имя
3 город  принять  2001 год  конец
4 село  война  январь  конец
5 город  война  январь  май
6 группа  январь  конец  игра
7 мир  январь  занимать  название
8 мир  2001  занимать  название
9 работа  мочь  январь  занимать
10 некоторый  вид  мочь  конец
11 часть  несколько  больший  большой
12 россия  мочь  новый  всё
13 большой  2001  май  игра
14 страна  однако  всё  новый
15 город  январь  конец  игра
16 март  всё  новый  второй
17 война  2001 год  родиться  мир
18 игра  мочь  ещё  группа
19 страна  конец  затем  игра
20 всё  работа  новый  январь
21 второй  май  игра  известный
22 река  война  январь  конец
23 работа  2001  занимать  мочь
24 январь  принять  занимать  мочь
25 площадь  год составлять  январь  игра
26 село  2001 год  население  январь
27 день  январь  конец  игра
28 группа  январь  конец  игра
29 несколько  январь  занимать  мочь
30 история  январь  занимать  название
31 тело  на

# 10. Лучшие темы NMF и сравнение с LDA

Лучшие темы: 

1.  участие  принимать  медаль  2001 год
2.  год составлять  занимать  перепись  составлять
3.  олимпийский  игра  принимать  участие

Как вы можете заметить, лучшие темы довольно сильно перекликаются с лучшими темами для LDA, однако стоит заметить, что тут все очень зависит от выставленных параметров. Тем не менее, мне показалось, что на человеческий взгляд модели LDA показывали в целом результаты лучшем, чем NMF, что в принципе может быть связано с количеством ключей, которые выдает каждая модель, так как LDA предоставляла больше ключей